# Explicit Cache in Vertex AI

In [12]:
import os
from IPython.display import Markdown, display
from google import genai
from google.genai.types import (
    Content,
    CreateCachedContentConfig,
    GenerateContentConfig,
    Part,
)

PROJECT_ID = "jc-gcp-project"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "us-central1"  # @param {type:"string"}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

In [13]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
MODEL_ID = "gemini-2.5-flash"  

In [14]:
# Gets a list of blob's in the specified bucket and path (prefix).
from google.cloud import storage
BUCKET_NAME = "jc-project-250618"
DESTINATION_BLOB_PREFIX = "vertex-ai-cached-images/" # folder name in your bucket

storage_client = storage.Client(project=PROJECT_ID)
blobs = storage_client.list_blobs(BUCKET_NAME, prefix=DESTINATION_BLOB_PREFIX)

In [15]:
# Convert each file into a Part.from_uri object and make it a list.
# Because it is a PNG file, specify mime_type as 'image/png'.
image_parts = [
    Part.from_uri(
        file_uri=f"gs://{BUCKET_NAME}/{blob.name}",
        mime_type="image/png"
    )
    for blob in blobs if not blob.name.endswith('/') 
]
# Check the generated part list (optional) 
print(f"A total of {len(image_parts)}image file was found in GCS.")
# for part in image_parts:
#     print(part.file_data.file_uri)

A total of 15image file was found in GCS.


In [16]:
system_instruction = """
Look at the images you have, and answer the following questions in 1-2 sentences.
"""

# Pass the generated image_parts list to the parts.
cached_content = client.caches.create(
    model=MODEL_ID,
    config=CreateCachedContentConfig(
        contents=[
            Content(
                role="user",
                parts=image_parts,  # Place the generated image part list here.
            )
        ],
        system_instruction=system_instruction,
        ttl="3600s", # Time-to-live: caching time (sec)
    ),
)

print("\n✅ The cache was successfully created.")
print(f"cache name: {cached_content.name}")


✅ The cache was successfully created.
cache name: projects/78565683329/locations/us-central1/cachedContents/6080348367307669504


In [17]:
print(cached_content.name)
print(cached_content.model)
print(cached_content.create_time)
print(cached_content.expire_time)
print(cached_content.usage_metadata)

projects/78565683329/locations/us-central1/cachedContents/6080348367307669504
projects/jc-gcp-project/locations/us-central1/publishers/google/models/gemini-2.5-flash
2025-08-13 07:13:47.290142+00:00
2025-08-13 08:13:47.270174+00:00
audio_duration_seconds=None image_count=15 text_count=68 total_token_count=3892 video_duration_seconds=None


In [18]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the common products in pictures you have?",
    config=GenerateContentConfig(
        cached_content=cached_content.name,
    ),
)

display(Markdown(response.text))

The common products displayed in all the pictures are various types of chairs and seating furniture. These range from office chairs and armchairs to lounge chairs and small sofas designed for sitting.

In [19]:
# 1. local image to use my new request
from google.genai import types

local_image_path = "images/actor_01.png"

# 2. local image to binary
try:
    with open(local_image_path, "rb") as f:
        image_bytes = f.read()
except FileNotFoundError:
    print(f"오류: '{local_image_path}' No file, check the file route.")
    exit()

# 3. Create Part
image_part = types.Part.from_bytes(
    data=image_bytes,
    mime_type="image/png"
)


# 4. Change prompt - trial01
specific_instruction_text = """
What is the image I have attached now, not cached.
"""
instruction_part = Part.from_text(text=specific_instruction_text)

# 5. new contests
contents = [
    instruction_part,
    image_part,
]

In [20]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
    config=GenerateContentConfig(
        cached_content=cached_content.name,
    ),
)

display(Markdown(response.text))

The images you have attached display a variety of chairs, including several types of office chairs, lounge chairs, armchairs, an inflatable chair, and a children's chair. There is also one image of a smiling man among the collection.

In [21]:
# 6. Prompt Engineering
specific_instruction_text = """
You are an expert interior stylist who recommends furniture that matches a client's personal style.

- **Your Catalog (Cached Images):** The various furniture (chair) images from our previous conversation are the product list you can offer.
- **Your Client (Newly Attached Image):** The newly attached portrait photo is your client.

**Your Task:**
Analyze your client's photo to understand their style, mood, and personality. Then, from the chairs in your catalog, choose and recommend the one that would best suit your client's personal space.

You must explain your recommendation as if you are speaking directly to the client. Specifically connect which features of the person (e.g., their clothing, expression, posture) match well with the chair's design elements (e.g., its material, color, shape), and explain why.
"""
instruction_part = types.Part.from_text(text=specific_instruction_text)

# 7. Set new contents
contents = [
    instruction_part,
    image_part,
]

In [22]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,  # New prompt
    config=GenerateContentConfig(
        # Cache
        cached_content=cached_content.name,
    ),
)

display(Markdown(response.text))

Given your polished and elegant appearance, marked by your sharp suit and gentle, warm smile, I recommend the luxurious beige executive office chair (Image 14). Its plush cushioning and sophisticated high-back design perfectly complement your refined demeanor, offering both the comfort and professional aesthetic that seem to align with your personal style.

In [53]:
# 6. Prompt Engineering
specific_instruction_text = """
You are an expert consultant who analyzes a client's style and mood to recommend the optimal product.

### Input Data
- **Product Catalog (Cached Images):** A list of images for various products that can be recommended to the client.
- **Client for Analysis (Newly Attached Image):** A photo of the client who has requested a product recommendation.

### Task & Output Format
Create a 'Final Recommendation Report' according to the format below. **It is crucial that you include the filenames accurately.**

---

## Final Recommendation Report

### 1. Client Analysis
- **Analyzed File:** [short description of the file]
- **Detailed Client Description:** [Describe the client's style, expression, mood, attire, and other observed details here]

### 2. Recommended Product
- **Recommended Product Filename:** [Return file number in the cached]
- **Detailed Product Description:** [Describe the selected product's design, color, material, features, and other details here]

### 3. Justification for Recommendation
- **Matching Point Explanation:** [Logically explain why this product is the best match for the client by specifically connecting the analyzed client features with the recommended product features.]
"""
instruction_part = types.Part.from_text(text=specific_instruction_text)

# 7. Set new contents
contents = [
    instruction_part,
    image_part,
]

In [54]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,  # New prompt
    config=GenerateContentConfig(
        # Cache
        cached_content=cached_content.name,
    ),
)

display(Markdown(response.text))

## Final Recommendation Report

### 1. Client Analysis
- **Analyzed File:** Image of a smiling East Asian male.
- **Detailed Client Description:** The client is a young, well-groomed East Asian male with a warm and approachable smile. He is dressed in formal or semi-formal attire, including a crisp white shirt and a dark suit jacket, suggesting a professional, refined, and classic sense of style. His overall mood appears pleasant and sophisticated.

### 2. Recommended Product
- **Recommended Product Filename:** `image_9.jpeg`
- **Detailed Product Description:** The recommended product is a tan upholstered armchair with a wooden frame, characteristic of mid-century modern design. It features clean lines, square tufted cushions for both the seat and back, and elegant wooden armrests and legs. The color is a warm, inviting tan, complementing the natural wood tones.

### 3. Justification for Recommendation
- **Matching Point Explanation:** The mid-century modern armchair in image_9.jpeg perfectly complements the client's refined, classic, and approachable style. Its clean lines and timeless design resonate with his sophisticated attire, while the warm tan upholstery and wooden elements offer an inviting comfort that mirrors his pleasant demeanor. This chair would fit seamlessly into a tastefully designed space, reflecting the client's elegant aesthetic.

In [55]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  cached_content_token_count=3892,
  candidates_token_count=273,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=273
    ),
  ],
  prompt_token_count=4400,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=272
    ),
    ModalityTokenCount(
      modality=<MediaModality.IMAGE: 'IMAGE'>,
      token_count=4128
    ),
  ],
  thoughts_token_count=1034,
  total_token_count=5707,
  traffic_type=<TrafficType.ON_DEMAND: 'ON_DEMAND'>
)